# Test notebook for Acceptance Test Campaign related to LSST Science Pipelines Release 20.0

This test will be executed on the LSST Science Platform Notebook Aspect, initialized with Science Pipelines release `r20-0-0`.

### Test case LVV-T39: Verify implementation of Generate Photometric Zeropoint for Visit Image
Verify that Processed Visit Image data products produced by the DRP and AP pipelines include the parameters of a model that relates the observed flux on the image to physical flux units.

In [ ]:
# Confirm that the version of the Science Pipelines is v20_0_0:
! echo $HOSTNAME
! eups list -s | grep lsst_distrib

In [ ]:
import lsst.daf.persistence as dafPersist

In [ ]:
import numpy as np
from astropy.table import Table
import random

We will use HSC-RC2, as processed using `v20_0_0_rc1`, which is the pipelines version that was used to create `v20_0_0`.

In [ ]:
# The output repo is tagged with the Jira ticket number "DM-25349":
rc2_repo = '/datasets/hsc/repo/rerun/RC/v20_0_0_rc1/DM-25349-sfm'
# Initialize the butler repo:
butler = dafPersist.Butler(rc2_repo)

In [ ]:
# Select an arbitrary visit image:
filters = ['HSC-G', 'HSC-R', 'HSC-I', 'HSC-Z', 'HSC-Y']
visits = [11696, 1210, 1238, 1172, 328]
# Coadd
tract = 9813
patch = '5,3'

The following selects a random CCD for one visit each from the _grizy_ filters. Then it does the following:
 1. Extract the instrumental flux at magnitude 0.0 (using photocalib.getInstFluxAtZeroMagnitude()).
 2. Extract the instrumental flux of a 23rd magnitude star.
 3. Convert the flux extracted in step 2 to a magnitude, and confirm that it returns 23.0.
 4. Convert the flux extracted in step 1 to a magnitude, and confirm that it returns 0.0.
 5. Convert the flux at magnitude zero to nanoJanskys, using photocalib.instFluxToNanojansky().

In [ ]:
flux_zero = []
flux_mag23 = []
mag23_from_flux = []
mag0_from_flux = []
mag0_nJy = []

for vis, filt in zip(visits, filters):
    dataId = {'filter':filt, 'visit':vis, 'ccd':30}
    photocalib = butler.get('calexp_photoCalib', dataId = dataId)
    flux_zero.append(photocalib.getInstFluxAtZeroMagnitude())
    flux_mag23.append(photocalib.magnitudeToInstFlux(23.0))
    mag23_from_flux.append(photocalib.instFluxToMagnitude(photocalib.magnitudeToInstFlux(23.0)))
    mag0_from_flux.append(photocalib.instFluxToMagnitude(photocalib.getInstFluxAtZeroMagnitude()))
    mag0_nJy.append(photocalib.instFluxToNanojansky(photocalib.getInstFluxAtZeroMagnitude()))
    
tab = Table([filters, visits, flux_zero, flux_mag23, mag23_from_flux, mag0_from_flux, mag0_nJy],
            names=('filter', 'visit', 'flux_zero', 'flux_mag23', 'mag23_from_flux', 'mag0_from_flux', 'mag0_nJy'))

In [ ]:
tab

In [ ]:
# Confirm that the instrumental flux at zero magnitude converts to
#  the same value (in nanoJanskys) for all filters:

cond = all(ele == mag0_nJy[0] for ele in mag0_nJy) 
if cond:
    print('All fluxes are equal.')
else:
    print('Physical fluxes for zero mag stars are NOT equal.')

In [ ]:
# Confirm that the instrumental flux for a 23rd magnitude star converts back to
#  mag=23.0:

cond = all(ele == 23.0 for ele in mag23_from_flux) 
if cond:
    print('All magnitudes are equal to 23.')
else:
    print('Fluxes for 23rd mag stars do NOT convert to mag=23.')

In [ ]:
# Confirm that the instrumental flux for a 23rd magnitude star converts back to
#  mag=23.0:

cond = all(ele == 0.0 for ele in mag0_from_flux) 
if cond:
    print('All magnitudes are equal to 0.')
else:
    print('Fluxes for zero mag stars do NOT convert to mag=0.')

### Select random sources, extract their magnitudes, and then confirm that they fall in a reasonable range (between roughly 12-32 mag for all objects)

In [ ]:
print(dataId)
src = butler.get('src', dataId = dataId)
photocalib = butler.get('calexp_photoCalib', dataId = dataId)

In [ ]:
random_mags = []

for i in range(500):
    rand_flux = random.choice(src['base_PsfFlux_instFlux'])
    random_mags.append(photocalib.instFluxToMagnitude(rand_flux))

In [ ]:
print('Brightest magnitude: ', np.nanmin(random_mags))
print('Faintest magnitude: ', np.nanmax(random_mags))
print('Median magnitude: ', np.nanmedian(random_mags))
print('Magnitude at 5th percentile: ', np.nanpercentile(random_mags, 5.0))
print('Magnitude at 95th percentile: ', np.nanpercentile(random_mags, 95.0))

The magnitudes are in a reasonable range, so we will call this a passing result -- the visit images have a zeropoint associated with them that allows for fluxes to be converted to magnitudes.

### Now do the same exercise, but for difference images from AP processing.

NOTE: we will only check a single image in this case.

In [ ]:
# Meredith pointed me to where she has run HSC data through diff imaging:
diffim_repo = '/project/mrawls/cosmos/rerun/ap_pipe-g-202007'

In [ ]:
butler_diffim = dafPersist.Butler(diffim_repo)

In [ ]:
# Pick a visit/ccd at random:
dataId_diffim = {'visit': 106078, 'ccd': 35}

In [ ]:
photocalib_diffim = butler_diffim.get('deepDiff_differenceExp_photoCalib', dataId=dataId_diffim)
diasrc = butler_diffim.get('deepDiff_diaSrc', **dataId_diffim)
deepDiff_mags = photocalib_diffim.instFluxToMagnitude(diasrc, 'ip_diffim_forced_PsfFlux')

flux_zero_tmp = photocalib.getInstFluxAtZeroMagnitude()
flux_mag23_tmp = (photocalib.magnitudeToInstFlux(23.0))
mag23_from_flux_tmp = (photocalib.instFluxToMagnitude(photocalib.magnitudeToInstFlux(23.0)))
mag0_from_flux_tmp = (photocalib.instFluxToMagnitude(photocalib.getInstFluxAtZeroMagnitude()))
mag0_nJy_tmp = (photocalib.instFluxToNanojansky(photocalib.getInstFluxAtZeroMagnitude()))

print('flux_zero: ', flux_zero_tmp)
print('flux_mag23: ', flux_mag23_tmp)
print('mag23_from_flux: ', mag23_from_flux_tmp)
print('mag0_from_flux: ', mag0_from_flux_tmp)
print('mag0_nJy: ', mag0_nJy_tmp)

In [ ]:
random_mags = []

for i in range(500):
    rand_flux = random.choice(diasrc['ip_diffim_forced_PsfFlux_instFlux'])
    random_mags.append(photocalib_diffim.instFluxToMagnitude(rand_flux))

In [ ]:
print('Brightest magnitude: ', np.nanmin(random_mags))
print('Faintest magnitude: ', np.nanmax(random_mags))
print('Median magnitude: ', np.nanmedian(random_mags))
print('Magnitude at 5th percentile: ', np.nanpercentile(random_mags, 5.0))
print('Magnitude at 95th percentile: ', np.nanpercentile(random_mags, 95.0))

The test passes - the magnitudes for difference sources have reasonable values.